In [1]:
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import pandas as pd
import shutil
import time

In [2]:
# define folder paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")

# open rgi df
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
# get all the data for the 5-year average products (infilled and not infilled)
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Average AAs')

# get list of all the .tif AAs in the folder
all_tifs_names = os.listdir(folder_products)

# keep only unfilled tif file paths, make them into full paths
all_tifs_names = [i for i in all_tifs_names if i[-9:]=='final.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names] # get rgiid

# make dfs
df = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':12345})

# keep only unfilled tif file paths, make them into full paths
all_tifs_names = os.listdir(folder_products)
all_tifs_names = [i for i in all_tifs_names if i[-10:]=='infill.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names] # get rgiid
df_infill = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':99999})

### add in the products for each individual year
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Annual AAs')
all_tifs_names = os.listdir(folder_products)
all_tifs_names = [i for i in all_tifs_names if i[-9:]=='final.tif']
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]
all_rgi_id = [i[3:17] for i in all_tifs_names]
all_years = [int(i[18:22]) for i in all_tifs_names]
df2 = pd.DataFrame({'RGIId':all_rgi_id, 'tif_name':all_tifs_names, 'tif_path':all_tifs_paths, 'year':all_years})

# concat
df_all = pd.concat([df, df_infill, df2])
# df_all = df

# add o2 region as column
df_all = df_all.merge(rgi_gdf, on='RGIId', how='left')
print(len(df))
print(len(df_infill))
print(len(df2))
print(len(df_all))
df_all
# df_all['O2Region'].unique()

3002
3002
14900
20904


,RGIId,tif_name,tif_path,year,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,...,Status,Connect,Form,TermType,Surging,Linkages,Name,O3Region,O3_name,geometry
0,RGI60-01.00004,S2_RGI60-01.00004_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213880E63381N,20090703,-9999999,-146.120,63.3810,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((390852.892 1513270.533, 390859.011 1..."
1,RGI60-01.00005,S2_RGI60-01.00005_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G212943E63551N,20090703,-9999999,-147.057,63.5510,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((344516.095 1527311.059, 344445.809 1..."
2,RGI60-01.00006,S2_RGI60-01.00006_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213756E63571N,20090703,-9999999,-146.244,63.5710,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((383858.051 1532357.663, 383852.553 1..."
3,RGI60-01.00010,S2_RGI60-01.00010_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G212830E63513N,20090703,-9999999,-147.170,63.5130,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((339736.019 1522275.072, 339760.103 1..."
4,RGI60-01.00012,S2_RGI60-01.00012_2018_2022_average_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,12345,G213349E63366N,20090703,-9999999,-146.651,63.3660,1,...,0,0,0,0,9,9,NaN,5,Eastern Alaska Range,"POLYGON ((367276.652 1509860.919, 367276.151 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20899,RGI60-01.27110,S2_RGI60-01.27110_2018_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2018,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
20900,RGI60-01.27110,S2_RGI60-01.27110_2019_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2019,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
20901,RGI60-01.27110,S2_RGI60-01.27110_2020_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2020,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."
20902,RGI60-01.27110,S2_RGI60-01.27110_2021_AA_final.tif,C:\Users\lzell\OneDrive - Colostate\Desktop\AG...,2021,G230488E54596N,20049999,20069999,-129.512,54.5959,1,...,0,0,0,0,9,0,NaN,16,Southwestern Coast Mountains,"POLYGON ((1548637.484 799452.208, 1548552.087 ..."


In [4]:
# choose what year to do this for
ys = [12345, 99999, 2018,2019,2020,2021,2022]
labels = ["all", "all_infill", "2018","2019","2020","2021","2022"]

ys = [99999]
labels = ['all_infill']

for i in range(len(ys)):
    for n in [2,3,4,5,6]:
        
        label = labels[i]
        y = ys[i]
        print(label,n)

        # keep only the rgi ids that are in o2region x and the given years
        df_subset = df_all[ (df_all['O2Region']==str(n)) & (df_all['year']==y) ]
        print(len(df_subset))

        # grab the full list of paths
        all_paths = df_subset['tif_path'].values

        # define path for vrt, output location/name
        out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA.tif")
        vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'merged{n}_{label}.vrt')

        # build vrt dataset
        vrt = gdal.BuildVRT(vrt_path, list(all_paths), VRTNodata='nan', srcNodata=0)    
        vrt = 0

print("Done")

all_infill 2
656


C:\Users\lzell\anaconda3\envs\AGVA_env\lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


all_infill 3
176
all_infill 4
483
all_infill 5
541
all_infill 6
1146
Done


In [5]:
for i in range(len(ys)):
    for n in [2,3,4,5,6]:
        
        label = labels[i]
        y = ys[i]
        print(label,n)
        
        # define path for vrt, output location/name
        out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f"O2Region_0{n}_{label}_AA.tif")
        vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Mosaics', f'merged{n}_{label}.vrt')

        # Open the VRT dataset
        ds = gdal.Open(vrt_path)

        print(f"Mosaicing Region {n}")
        gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=2','COMPRESS=DEFLATE', 'ZLEVEL=9'])
        print("mosaicing done")
        
print("All done!")

all_infill 2
Mosaicing Region 2
mosaicing done
all_infill 3
Mosaicing Region 3
mosaicing done
all_infill 4
Mosaicing Region 4
mosaicing done
all_infill 5
Mosaicing Region 5
mosaicing done
all_infill 6
Mosaicing Region 6
mosaicing done
All done!
